In [11]:
# using Distributions 

# abstract type Distribution{T} end
# struct Half_Cauchy <: Distribution{Float64} end
# const half_cauchy = Half_Cauchy()

# function logpdf(::Half_Cauchy, x::Real, x0::Real, gamma::Real)
#     if x<0
#             x = x*-1
#     end
            
#     return Distributions.logpdf(Distributions.Cauchy(x0, gamma), x)
    
# end

# function logpdf_grad(::Half_Cauchy, x::Real, x0::Real, gamma::Real)
#     if x<0
#             x = x*-1
#     end
            
#     x_x0 = x - x0
#     x_x0_sq = x_x0^2
#     gamma_sq = gamma^2
#     deriv_x0 =  2 * x_x0 / (gamma_sq + x_x0_sq)
#     deriv_x = - deriv_x0
#     deriv_gamma = (x_x0_sq - gamma_sq) / (gamma * (gamma_sq + x_x0_sq))
#     (deriv_x, deriv_x0, deriv_gamma)
# end

# is_discrete(::Half_Cauchy) = false

# random(::Half_Cauchy, x0::Real, gamma::Real) = abs(rand(Distributions.Cauchy(x0, gamma)))

# (::Half_Cauchy)(x0::Real, gamma::Real) = random(Half_Cauchy(), x0, gamma)

# has_output_grad(::Half_Cauchy) = true
# has_argument_grads(::Half_Cauchy) = (true, true)

# export half_cauchy


In [12]:
# using  Gen

# @gen function a()
#     x ~ half_cauchy(0,1)

# end

# traces = [simulate(a, ()) for _ in 1:2]  # Run 100 importance samples

In [14]:
using Gen
using Plots

using Distributions 




#my_dist = print(rand(half_cauchy(0,25)))
#my_dist = Truncated(Cauchy(), 0, Inf)





# Define the model with dynamic eta sampling
@gen function simple_normal_model(sigma)
    mu ~ normal(0, 1)     # Sample mu from a normal distribution
    tau  ~ normal(2, 5)   # Sample tau from a Half-Cauchy distribution
    
    # Dynamic eta values sampled from normal distributions
    list_of_Eta = [{(:eta, i)} ~ normal(0, 1) for i=1:length(sigma)]
    
    for i in 1:5  # Loop over 5 iterations
        # Calculate theta based on mu, tau, and eta
        theta = mu + tau * list_of_Eta[i]
        
        # Sample obs from a normal distribution with mean theta and standard deviation sigma[i]
        {(:obs, i)} ~ normal(theta, sigma[i])
    end

    return :obs
    
    
end

function do_inference(model, xs, ys, amount_of_computation)
    
    # Create a choice map that maps model addresses (:y, i)
    # to observed values ys[i]. We leave :slope and :intercept
    # unconstrained, because we want them to be inferred.
    observations = Gen.choicemap()
    for (i, y) in enumerate(ys)
        observations[(:y, i)] = y
    end
    
    # Call importance_resampling to obtain a likely trace consistent
    # with our observations.
    (trace, _) = Gen.importance_resampling(model, (xs,), observations, amount_of_computation);
    return trace
end;

y_obs = [28, 8, -3, 7, -1, 1, 18, 12]
sigma =[15, 10, 16, 11, 9, 11, 10, 18]


# Use importance sampling as an example:
traces = [simulate(simple_normal_model, (sigma,)) for _ in 1:100]  # Run 100 importance samples

trace = do_inference(simple_normal_model, sigma, y_obs, 1000)

# Extract mu values from the traces
mu = trace[:mu]

print(mu)


-0.2693454787568785